In [126]:
import pandas as pd
import requests
from tqdm.notebook import tqdm as tqdm_notebook

In [127]:
URL = "https://api.dragoncapital.com.vn/fundfactsheet/top_holdings/getLatestValue.php"

In [128]:
codes = ["DCDE", "DCDS"]

In [129]:
def get_history(code: str, dates: list[str]):
    history: list[dict] = []
    for date_upload in tqdm_notebook(dates):
        url: str = f"{URL}?trade_code={code}&date_upload={date_upload}"
        response: requests.Response = requests.get(url)
        data: dict = response.json()
        ffs_holding: list[dict] = data.get("ffs_holding", [])
        ffs_holding = list(
            map(lambda a: {**a, "code": code, "date": date_upload}, ffs_holding)
        )
        history: list[dict] = history + ffs_holding
    return history

In [130]:
def get_product(code: str):
    product_url = f"{URL}?trade_code={code}"
    response = requests.get(product_url)
    product: dict = response.json()
    available_dates: list[str] = product.get("available_dates", [])
    history = get_history(code, available_dates)
    return history

In [133]:
history = []

for code in codes:
    history_product = get_product(code)
    history = history + history_product

history_data_frame = pd.DataFrame(history)
history_data_frame = history_data_frame[
    ["code", "date", "stock", "bourse_en", "per_nav"]
]
history_data_frame = history_data_frame.rename(
    columns={"bourse_en": "market", "per_nav": "percentage"}
)
history_data_frame.to_csv("./history.csv", index=False)

  0%|          | 0/93 [00:00<?, ?it/s]

  0%|          | 0/97 [00:00<?, ?it/s]